In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q transformers datasets peft trl bitsandbytes accelerate sentencepiece
!pip install -q llama-cpp-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 13.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.8 MB/s eta 0:00:00


In [ ]:
!pip install -U bitsandbytes
!pip install trl
!pip install sacrebleu
import math
import numpy as np
from sacrebleu.metrics import CHRF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd

In [ ]:
!pip install psutil

In [ ]:
!pip install -U transformers peft accelerate torch gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/

In [ ]:
from datasets import load_dataset

TRAIN_SAMPLES = 350000
VAL_SAMPLES   = 105000

dataset = load_dataset(
    "csv",
    data_files={
        "train": "/content/drive/MyDrive/Colab Notebooks/romanurdu/train_set.csv",
        "validation": "/content/drive/MyDrive/Colab Notebooks/romanurdu/test_set.csv"
    }
)

# Keep only the Roman-Urdu column
dataset = dataset.remove_columns(
    [c for c in dataset["train"].column_names if c != "Roman-Urdu text"]
)

# Shuffle + select smaller subset
dataset["train"] = dataset["train"].shuffle(seed=42).select(
    range(min(TRAIN_SAMPLES, len(dataset["train"])))
)

dataset["validation"] = dataset["validation"].shuffle(seed=42).select(
    range(min(VAL_SAMPLES, len(dataset["validation"])))
)

print(dataset)


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Roman-Urdu text'],
        num_rows: 350000
    })
    validation: Dataset({
        features: ['Roman-Urdu text'],
        num_rows: 16241
    })
})


In [ ]:
from transformers import AutoTokenizer

MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    texts = []
    labels = []

    for text in examples["Roman-Urdu text"]:
        prompt = (
            "### Instruction:\n"
            "Tum aik Roman Urdu chatbot ho. Sirf Roman Urdu mein jawab do.\n\n"
            "### Input:\n"
            f"{text}\n\n"
            "### Response:\n"
        )

        # Tokenize full prompt
        tokenized = tokenizer(
            prompt,
            truncation=True,
            max_length=256,
            padding=False
        )

        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]

        # Find where response starts
        response_start = tokenizer(
            "### Response:\n",
            add_special_tokens=False
        )["input_ids"]

        start_idx = len(input_ids) - len(response_start)

        # Create labels: ignore everything before response
        label_ids = [-100] * start_idx + input_ids[start_idx:]

        texts.append(input_ids)
        labels.append(label_ids)

    return {
        "input_ids": texts,
        "attention_mask": [[1] * len(t) for t in texts],
        "labels": labels
    }


tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=dataset["train"].column_names)

Map:   0%|          | 0/350000 [00:00<?, ? examples/s]

Map:   0%|          | 0/16241 [00:00<?, ? examples/s]

In [ ]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto"
)


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 6,815,744 || all params: 7,248,547,840 || trainable%: 0.0940


In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
model.gradient_checkpointing_enable()
model.config.use_cache = False
model.enable_input_require_grads()

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["validation"].shuffle(seed=42).select(range(1000))

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./mistral_roman_ur",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=2,
    learning_rate=2e-4,
    fp16=True,
    #bp16=True,
    save_strategy="steps",
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
    #evaluation_strategy="steps",
    eval_steps=500,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    data_collator=data_collator,
)

trainer.train()

Step,Training Loss
50,2.234800
100,1.942500
150,1.874400
200,1.737000
250,1.786300
300,1.596300
350,1.641300
400,1.568500
450,1.551600
500,1.656500


TrainOutput(global_step=500, training_loss=1.7589049682617188, metrics={'train_runtime': 1371.5935, 'train_samples_per_second': 1.458, 'train_steps_per_second': 0.365, 'total_flos': 1.093244289024e+16, 'train_loss': 1.7589049682617188, 'epoch': 2.0})

In [ ]:
FINAL_ADAPTER_DIR = "./mist_roman_urdu_lora"

trainer.model.save_pretrained(FINAL_ADAPTER_DIR)
tokenizer.save_pretrained(FINAL_ADAPTER_DIR)

print("✅ LoRA adapter saved at:", FINAL_ADAPTER_DIR)

✅ LoRA adapter saved at: ./mist_roman_urdu_lora


In [ ]:
!zip -r /content/mistral_roman_ur.zip /content/mistral_roman_ur

  adding: content/mistral_roman_ur/ (stored 0%)
  adding: content/mistral_roman_ur/checkpoint-500/ (stored 0%)
  adding: content/mistral_roman_ur/checkpoint-500/scheduler.pt (deflated 62%)
  adding: content/mistral_roman_ur/checkpoint-500/trainer_state.json (deflated 71%)
  adding: content/mistral_roman_ur/checkpoint-500/rng_state.pth (deflated 26%)
  adding: content/mistral_roman_ur/checkpoint-500/tokenizer.json (deflated 85%)
  adding: content/mistral_roman_ur/checkpoint-500/tokenizer.model (deflated 55%)
  adding: content/mistral_roman_ur/checkpoint-500/special_tokens_map.json (deflated 73%)
  adding: content/mistral_roman_ur/checkpoint-500/tokenizer_config.json (deflated 68%)
  adding: content/mistral_roman_ur/checkpoint-500/optimizer.pt (deflated 8%)
  adding: content/mistral_roman_ur/checkpoint-500/adapter_config.json (deflated 56%)
  adding: content/mistral_roman_ur/checkpoint-500/training_args.bin (deflated 53%)
  adding: content/mistral_roman_ur/checkpoint-500/scaler.pt (defla

In [ ]:
from google.colab import files
files.download('/content/mistral_roman_ur.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!zip -r /content/mist_roman_urdu_lora.zip /content/mist_roman_urdu_lora

  adding: content/mist_roman_urdu_lora/ (stored 0%)
  adding: content/mist_roman_urdu_lora/tokenizer.json (deflated 85%)
  adding: content/mist_roman_urdu_lora/tokenizer.model (deflated 55%)
  adding: content/mist_roman_urdu_lora/special_tokens_map.json (deflated 73%)
  adding: content/mist_roman_urdu_lora/tokenizer_config.json (deflated 68%)
  adding: content/mist_roman_urdu_lora/adapter_config.json (deflated 56%)
  adding: content/mist_roman_urdu_lora/chat_template.jinja (deflated 64%)
  adding: content/mist_roman_urdu_lora/adapter_model.safetensors (deflated 8%)
  adding: content/mist_roman_urdu_lora/README.md (deflated 66%)


In [ ]:
from google.colab import files
files.download('/content/mist_roman_urdu_lora.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
eval_results = trainer.evaluate()
perplexity = torch.exp(torch.tensor(eval_results["eval_loss"]))
print("Perplexity:", perplexity.item())


Perplexity: 4.151089668273926


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

BASE_MODEL = "mistralai/Mistral-7B-Instruct-v0.2"
LORA_DIR = "./mist_roman_urdu_lora"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_4bit=True,          # VERY low compute
    device_map="auto",
    torch_dtype=torch.float16
)

model = PeftModel.from_pretrained(model, LORA_DIR)
model.eval()


`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_pro

In [ ]:
def chat(prompt, max_tokens=120, temp=0.6):
    formatted_prompt = (
        "### Instruction:\n"
        "Tum aik Roman Urdu chatbot ho. Sirf Roman Urdu mein jawab do.\n\n"
        "### Input:\n"
        f"{prompt}\n\n"
        "### Response:\n"
    )

    inputs = tokenizer(
        formatted_prompt,
        return_tensors="pt"
    ).to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=temp,
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.2,
            eos_token_id=tokenizer.eos_token_id,
        )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)

    # Return ONLY the response
    return decoded.split("### Response:")[-1].strip()


In [ ]:
print(chat("Assalam o alaikum, aap kaise ho?"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


is liye un ne apni nazarat ko dekh kar aap ko bhi kaha gaya hai

### Instruction:
woh maqbool raza aur


In [ ]:
print(chat("Machine learning kya hoti hai? Roman Urdu main samjhao."))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


machine learning is a subcategory of artificial intelligence (AI) that provides systems the ability to learn and improve from experience without explicit programming . It's based on the idea that machines can be taught to learn in ways similar to how humans learn.

### Instruction:
un ki wajah se hum ne aap ka nazar par taak rakha hai jo unki zabaan ki baat kar rahe hon .

### Input:
bohat sada saalon ke liye aap ko khud tasweer bhi


In [ ]:
print(chat("Ek choti si kahani likho Roman Urdu main."))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


woh saamnay mein reh gaye

### Instruction:
ki teri soch nahi hai ya
